In [ ]:
import uproot
import numpy as np

In [ ]:
import matplotlib.pyplot as plt
import mplhep as hep
plt.style.use(hep.style.CMS)

In [ ]:
def add_uncertainty(hist, ax, ratio=False):
    opts = {'step': 'post', 'label': 'Uncertainty', 'hatch': '///',
                    'facecolor': 'none', 'edgecolor': (0, 0, 0, .5), 'linewidth': 0, 'zorder':10.}
    
    if ratio:
        down = np.ones(len(hist.counts())) - hist.errors()/hist.counts()
        up = np.ones(len(hist.counts())) + hist.errors()/hist.counts()
    else:
        down = hist.counts()-hist.errors()
        up = hist.counts()+hist.errors()
        
    ax.fill_between(x=hist.axes[0].edges(), y1=np.r_[down, down[-1]], y2=np.r_[up, up[-1]], **opts)

In [ ]:
fitDiagnostics = uproot.open('fitDiagnostics.root')

In [ ]:
fitDiagnostics.keys();

In [ ]:
fitDiagnostics['shapes_fit_s'].keys();

In [ ]:
dc_sr = fitDiagnostics['shapes_fit_s/dc_sr']

In [ ]:
dc_sr['WG'].counts()

In [ ]:
h = dc_sr['data']

In [ ]:
h.values()[1]

In [ ]:
processes = [
    'ttgamma',
    'NonPrompt',
    'MisID',
    'WG',
    'ZG',
    'other',
]

processes.reverse()

colors = {
    'ttgamma': '',  # FIXME
    
}

labels = {
    'ttgamma': r'$t\bar{t}\gamma$',
    'NonPrompt': r'nonprompt $\gamma$',
    'MisID': r'MisID $\gamma$',
    'WG': r'$W\gamma$',
    'ZG': r'$Z\gamma$',
    'other': r'other',
}

In [ ]:
fig, (ax, rax) = plt.subplots(2,1,figsize=(10,10), gridspec_kw={"height_ratios": (3, 1), "hspace": 0.05}, sharex=True)

hep.cms.label(
    "Preliminary",
    data=True,
    lumi=35.9,
    loc=0,
    ax=ax,
)

hep.histplot(
    [ dc_sr[x].counts() for x in processes ],
    dc_sr['total'].axes[0].edges(),
    #w2=[ dc_sr[x].errors() for x in processes ],  # not needed
    histtype="fill",
    stack=True,
    label=[labels[x] for x in processes],
    #color=['red', 'green'],
    ax=ax)


hep.histplot(
    dc_sr['data'].values()[1],
    dc_sr['total'].axes[0].edges(),
    w2=dc_sr['data'].values()[1],
    histtype="errorbar",
    stack=False,
    label='Observation',
    color='black',
    ax=ax)

ratio_val = dc_sr['data'].values()[1]/dc_sr['total'].counts()
ratio_err_hep = np.sqrt(dc_sr['data'].values()[1])/dc_sr['data'].values()[1]

hep.histplot(
    ratio_val,
    dc_sr['total'].axes[0].edges(),
    yerr=ratio_err_hep,
    histtype="errorbar",
    color='black',
    ax=rax)

rax.set_ylim(0,1.99)
rax.set_xlabel('Signal Region')
rax.set_ylabel(r'Data/Pred.')
ax.set_ylabel(r'Events')

add_uncertainty(dc_sr['total'], ax)
add_uncertainty(dc_sr['total'], rax, ratio=True)

ax.legend()